In [1]:
import plotly.express as px

In [2]:
import numpy as np

In [6]:
data=np.loadtxt('ecg 100.dat',skiprows=2)

In [7]:
data

array([[ 0.0000e+00, -1.4500e-01, -6.5000e-02],
       [ 3.0000e-03, -1.4500e-01, -6.5000e-02],
       [ 6.0000e-03, -1.4500e-01, -6.5000e-02],
       ...,
       [ 5.9992e+01, -2.3000e-01, -1.7500e-01],
       [ 5.9994e+01, -2.4500e-01, -1.8000e-01],
       [ 5.9997e+01, -2.4500e-01, -1.7500e-01]])

In [9]:
data[:,:2]

array([[ 0.0000e+00, -1.4500e-01],
       [ 3.0000e-03, -1.4500e-01],
       [ 6.0000e-03, -1.4500e-01],
       ...,
       [ 5.9992e+01, -2.3000e-01],
       [ 5.9994e+01, -2.4500e-01],
       [ 5.9997e+01, -2.4500e-01]])

In [10]:
x= data[:,0]
y= data[:,1]

In [11]:
x

array([0.0000e+00, 3.0000e-03, 6.0000e-03, ..., 5.9992e+01, 5.9994e+01,
       5.9997e+01])

In [12]:
y

array([-0.145, -0.145, -0.145, ..., -0.23 , -0.245, -0.245])

In [13]:
px.line(x=x,y=y)

In [29]:
def FFT_vectorized(x):
    """A vectorized, non-recursive version of the Cooley-Tukey FFT"""
    x = np.asarray(x, dtype=float)
    N = x.shape[0]

    if np.log2(N) % 1 > 0:
        raise ValueError("size of x must be a power of 2")

    # N_min here is equivalent to the stopping condition above,
    # and should be a power of 2
    N_min = min(N, 32)
    
    # Perform an O[N^2] DFT on all length-N_min sub-problems at once
    n = np.arange(N_min)
    k = n[:, None]
    M = np.exp(-2j * np.pi * n * k / N_min)
    X = np.dot(M, x.reshape((N_min, -1)))

    # build-up each level of the recursive calculation all at once
    while X.shape[0] < N:
        X_even = X[:, :X.shape[1] / 2]
        X_odd = X[:, X.shape[1] / 2:]
        factor = np.exp(-1j * np.pi * np.arange(X.shape[0])
                        / X.shape[0])[:, None]
        X = np.vstack([X_even + factor * X_odd,
                       X_even - factor * X_odd])

    return X.ravel()

In [33]:
from icecream import ic
def DFT_slow(x):
    """Compute the discrete Fourier Transform of the 1D array x"""
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    n = np.arange(N)
    k = n.reshape((N, 1))
    M = np.exp(-2j * np.pi * k * n / N)
    return np.dot(M, x)
def FFT(x):
    """A recursive implementation of the 1D Cooley-Tukey FFT"""
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    
    if N % 2 > 0:
        raise ValueError("size of x must be a power of 2")
    elif N <= 32:  # this cutoff should be optimized
        return DFT_slow(x)
    else:
        X_even = FFT(x[::2])
        X_odd = FFT(x[1::2])
        factor = np.exp(-2j * np.pi * np.arange(N) / N)
        return np.concatenate([X_even + factor[:N / 2] * X_odd,
                               X_even + factor[N / 2:] * X_odd])

In [34]:
a= np.random.random(1024)
ic(a)
FFT(a)


ic| a: array([0.12629522, 0.51000035, 0.94544968, ..., 0.88834435, 0.81399208,
              0.19430171])


NameError: name 'DFT_slow' is not defined